In [1]:
from tkinter import *
from random import choice

Реализуйте алгоритм генерации лабиринтов через поиск в ширину.
1. Реализуйте разные размеры лабиринтов.
2. Реализуйте экспорт получившихся лабиринтов во внешний файл.

In [ ]:
size = (20, 30, 30) #Задаём размер лабиринта (ширина, высота и размер клетки)

class maze(object): #Класс лабиринта, атрибуты: ширина, высота, матрица клеток
    def __init__(self, width, height, side, matrix):
        self.width = width
        self.height = height
        self.side = side
        self.matrix = matrix
        
#Класс клетки - будет храниться в матрице клеток лабиринта. Состоит из списка стен и цвета
class cell(object):
    def __init__(self, walls, colour):
        self.walls = walls
        self.colour = colour

#Создаём лабиринт (1 - стена есть, 0 - стены нет)
Maze = maze(size[0], size[1], size[2], [[cell([1,1,1,1], 'white') for j in range(size[1])] 
                                        for i in range(size[0])])
Maze.matrix[0][0].colour = 'yellow'

#Создаём холст (size[0] x size[2]) x (size[1] x size[2])
root = Tk()
canvas = Canvas(root, width = size[1]*size[2]+10, height = size[0]*size[2]+10)
canvas.pack()

#Функция создания лабиринта
Cursor = (Maze.matrix[0][0], 'yellow', (0,0)) #Избранная клетка, её цветом будет жёлтый
Way = [(0,0)] #Клетки, уже имеющиеся во всевозможных путях
Step_back = -2 #Шаг, на который мы будем уходить назад при тупике

def lab(cursor, way, mazze, sb):
    #Если лабиринт уже заполнен, то не даём жёлтому курсору попасть на экран
    if len(way) >= mazze.width*mazze.height:
        cursor[0].colour = 'green'
        
    #Визуализируем лабиринт
    canvas.delete('all')
    for h in range(mazze.height):
        for w in range(mazze.width):
            s = mazze.side
            #Рисуем клетки
            canvas.create_rectangle(5+h*s, 5+w*s, 5+(h+1)*s, 5+(w+1)*s, fill = mazze.matrix[w][h].colour,
                                   outline = mazze.matrix[w][h].colour)
            #Рисуем стены: лево, верх, право, низ
            if mazze.matrix[w][h].walls[0] == 1:
                canvas.create_rectangle(5+h*s, 5+w*s, 5+h*s, 5+(w+1)*s, fill = 'blue')
            if mazze.matrix[w][h].walls[1] == 1:
                canvas.create_rectangle(5+h*s, 5+w*s, 5+(h+1)*s, 5+w*s, fill = 'blue')
            if mazze.matrix[w][h].walls[2] == 1:
                canvas.create_rectangle(5+(h+1)*s, 5+w*s, 5+(h+1)*s, 5+(w+1)*s, fill = 'blue')
            if mazze.matrix[w][h].walls[3] == 1:
                canvas.create_rectangle(5+h*s, 5+(w+1)*s, 5+(h+1)*s, 5+(w+1)*s, fill = 'blue')
        
    #Задаём список соседей избранной клетки
    neighbors = []
    if cursor[2][0]-1 >= 0 and (cursor[2][0]-1, cursor[2][1]) not in way:
        neighbors.append((mazze.matrix[cursor[2][0]-1][cursor[2][1]], 'yellow', 
                          (cursor[2][0]-1, cursor[2][1]), 'up'))
    if cursor[2][1]-1 >= 0 and (cursor[2][0], cursor[2][1]-1) not in way:
        neighbors.append((mazze.matrix[cursor[2][0]][cursor[2][1]-1], 'yellow', 
                          (cursor[2][0], cursor[2][1]-1), 'left'))
    if cursor[2][0]+1 <= mazze.width-1 and (cursor[2][0]+1, cursor[2][1]) not in way:
        neighbors.append((mazze.matrix[cursor[2][0]+1][cursor[2][1]], 'yellow', 
                          (cursor[2][0]+1, cursor[2][1]), 'down'))
    if cursor[2][1]+1 <= mazze.height-1 and (cursor[2][0], cursor[2][1]+1) not in way:
        neighbors.append((mazze.matrix[cursor[2][0]][cursor[2][1]+1], 'yellow', 
                          (cursor[2][0], cursor[2][1]+1), 'right'))
    
    #Выбираем соседа, убираем стены, ставим "курсором" новую клетку
    if len(neighbors) > 0:
        sb = -2
        new_cursor = choice(neighbors)
        if new_cursor[3] == 'up':
            cursor[0].walls[1], new_cursor[0].walls[3] = 0, 0
        elif new_cursor[3] == 'left':
            cursor[0].walls[0], new_cursor[0].walls[2] = 0, 0
        elif new_cursor[3] == 'down':
            cursor[0].walls[3], new_cursor[0].walls[1] = 0, 0
        else:
            cursor[0].walls[2], new_cursor[0].walls[0] = 0, 0
        cursor[0].colour = 'green'
        way.append(new_cursor[2])
        cursor = new_cursor
        new_cursor[0].colour = 'yellow'
    
    #Если соседей нет, то движемся назад по пути
    else:
        cursor[0].colour = 'green'
        cursor = (Maze.matrix[way[sb][0]][way[sb][1]], 'yellow', (way[sb][0],way[sb][1]))
        cursor[0].colour = 'yellow'
        if len(way) != mazze.width*mazze.height:
            sb -= 1
    
    #Экспортируем лабиринт, если он заполнен
    if len(way) == mazze.width*mazze.height:
        canvas.postscript(file = 'Maze.eps')
        
    root.after(10, lab, cursor, way, mazze, sb)

lab(Cursor, Way, Maze, Step_back)

root.mainloop()